# Proyek Akhir: Menyelesaikan Permasalahan Perusahaan Jaya Jaya Maju

- Nama: Muhammad Hafizh Dzaki
- Email: muhammadhafizhdzaki@gmail.com
- Id Dicoding: haztsu

## Persiapan

### Menyiapkan library yang dibutuhkan

In [35]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import statsmodels.api as sm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from imblearn.over_sampling import SMOTE, RandomOverSampler

### Menyiapkan data yang akan diguankan

## Data Understanding

In [36]:
df = pd.read_csv('employee_data.csv')
df.head()

,EmployeeId,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1,38,NaN,Travel_Frequently,1444,Human Resources,1,4,Other,1,...,2,80,1,7,2,3,6,2,1,2
1,2,37,1.0,Travel_Rarely,1141,Research & Development,11,2,Medical,1,...,1,80,0,15,2,1,1,0,0,0
2,3,51,1.0,Travel_Rarely,1323,Research & Development,4,4,Life Sciences,1,...,3,80,3,18,2,4,10,0,2,7
3,4,42,0.0,Travel_Frequently,555,Sales,26,3,Marketing,1,...,4,80,1,23,2,4,20,4,4,8
4,5,40,NaN,Travel_Rarely,1194,Research & Development,2,4,Medical,1,...,2,80,3,20,2,3,5,3,0,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmployeeId                1470 non-null   int64  
 1   Age                       1470 non-null   int64  
 2   Attrition                 1058 non-null   float64
 3   BusinessTravel            1470 non-null   object 
 4   DailyRate                 1470 non-null   int64  
 5   Department                1470 non-null   object 
 6   DistanceFromHome          1470 non-null   int64  
 7   Education                 1470 non-null   int64  
 8   EducationField            1470 non-null   object 
 9   EmployeeCount             1470 non-null   int64  
 10  EnvironmentSatisfaction   1470 non-null   int64  
 11  Gender                    1470 non-null   object 
 12  HourlyRate                1470 non-null   int64  
 13  JobInvolvement            1470 non-null   int64  
 14  JobLevel

In [4]:
df.nunique()[df.nunique() < 2]

EmployeeCount    1
Over18           1
StandardHours    1
dtype: int64

In [5]:
df.columns

Index(['EmployeeId', 'Age', 'Attrition', 'BusinessTravel', 'DailyRate',
       'Department', 'DistanceFromHome', 'Education', 'EducationField',
       'EmployeeCount', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [6]:
unnecessary_features = ['EmployeeId', 'EmployeeCount', 'Over18', 'StandardHours']
numerical_features = ['Age', 'DailyRate', 'DistanceFromHome', 
                      'HourlyRate', 'MonthlyIncome', 'MonthlyRate', 
                      'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears', 
                      'TrainingTimesLastYear', 'YearsAtCompany', 'YearsInCurrentRole', 
                      'YearsSinceLastPromotion', 'YearsWithCurrManager']
categorical_features = ['BusinessTravel', 
                        'Department', 'Education', 'EducationField',
                        'EnvironmentSatisfaction', 'Gender', 'JobInvolvement',
                        'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 
                        'OverTime', 'PerformanceRating', 'RelationshipSatisfaction', 
                        'StockOptionLevel', 'WorkLifeBalance']

### Berdasarkan riset dan artikel scientific yang ada sebelumnya, terdapat beberapa faktor yang memungkinkan karyawan ingin *attrition*


1. Hal Terkait Kompensasi: <br>MonthlyIncome, PercentSalaryHike, StockOptionLevel
    
2. Hal Terkait Individu: <br> Age, DistanceFromHome, Education, EducationField, Gender, JobInvolvement, JobRole, MaritalStatus, WorkLifeBalance

3. Hal Terkait Perusahaan: <br>BusinessTravel, Department, EnvironmentSatisfaction, JobSatisfaction, OverTime, PerformanceRating, RelationshipSatisfaction

4. Hal Terkait Perkembangan Karir: <br>NumCompaniesWorked, TotalWorkingYears, JobLevel, TrainingTimesLastYear, YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager


Sumber bacaan:
- [Together Platform](https://www.togetherplatform.com/blog/attrition-rate)
- [Personio](https://www.personio.com/hr-lexicon/attrition-rate/) 
- [Linkedin Post - IBM HR Analystics](https://www.linkedin.com/pulse/ibm-hr-analytics-understanding-employee-data-christy/)

In [7]:
compensation_matters = ['MonthlyIncome', 'PercentSalaryHike', 'StockOptionLevel']
individuals_matters = ['Age', 'DistanceFromHome', 'Education', 'EducationField', 
                       'Gender', 'JobInvolvement', 'JobRole', 'MaritalStatus', 
                       'PerformanceRating', 'WorkLifeBalance']
company_matters = ['BusinessTravel', 'Department', 'EnvironmentSatisfaction', 
                   'JobSatisfaction', 'OverTime', 
                   'RelationshipSatisfaction']
carrer_growth_matters = ['NumCompaniesWorked', 'TotalWorkingYears', 'JobLevel', 
                         'TrainingTimesLastYear', 'YearsAtCompany', 'YearsInCurrentRole', 
                         'YearsSinceLastPromotion', 'YearsWithCurrManager']

## Data Preparation / Preprocessing

In [8]:
df = df.drop(unnecessary_features, axis=1)
df[categorical_features] = df[categorical_features].astype('category')
df[numerical_features] = df[numerical_features].astype('float64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Age                       1470 non-null   float64 
 1   Attrition                 1058 non-null   float64 
 2   BusinessTravel            1470 non-null   category
 3   DailyRate                 1470 non-null   float64 
 4   Department                1470 non-null   category
 5   DistanceFromHome          1470 non-null   float64 
 6   Education                 1470 non-null   category
 7   EducationField            1470 non-null   category
 8   EnvironmentSatisfaction   1470 non-null   category
 9   Gender                    1470 non-null   category
 10  HourlyRate                1470 non-null   float64 
 11  JobInvolvement            1470 non-null   category
 12  JobLevel                  1470 non-null   category
 13  JobRole                   1470 non-null   catego

In [9]:
print("Total null data:", df.isna().sum().sum())
df.dropna(inplace=True)
print("Total null data after dropping:", df.isna().sum().sum())
print("Total duplicate data:", df.duplicated().sum())
print("Total data after dropping null data:", df.shape[0], "rows &", df.shape[1], "columns") 

Total null data: 412
Total null data after dropping: 0
Total duplicate data: 0
Total data after dropping null data: 1058 rows & 31 columns


In [10]:
le = LabelEncoder()
for col in categorical_features:
    if df[col].dtype == 'category':
        df[col] = le.fit_transform(df[col])
        label_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        print(f"{col}:")
        for key, value in label_dict.items():
            print(f"  {key}: {value}")
        print("")

BusinessTravel:
  Non-Travel: 0
  Travel_Frequently: 1
  Travel_Rarely: 2

Department:
  Human Resources: 0
  Research & Development: 1
  Sales: 2

Education:
  1: 0
  2: 1
  3: 2
  4: 3
  5: 4

EducationField:
  Human Resources: 0
  Life Sciences: 1
  Marketing: 2
  Medical: 3
  Other: 4
  Technical Degree: 5

EnvironmentSatisfaction:
  1: 0
  2: 1
  3: 2
  4: 3

Gender:
  Female: 0
  Male: 1

JobInvolvement:
  1: 0
  2: 1
  3: 2
  4: 3

JobLevel:
  1: 0
  2: 1
  3: 2
  4: 3
  5: 4

JobRole:
  Healthcare Representative: 0
  Human Resources: 1
  Laboratory Technician: 2
  Manager: 3
  Manufacturing Director: 4
  Research Director: 5
  Research Scientist: 6
  Sales Executive: 7
  Sales Representative: 8

JobSatisfaction:
  1: 0
  2: 1
  3: 2
  4: 3

MaritalStatus:
  Divorced: 0
  Married: 1
  Single: 2

OverTime:
  No: 0
  Yes: 1

PerformanceRating:
  3: 0
  4: 1

RelationshipSatisfaction:
  1: 0
  2: 1
  3: 2
  4: 3

StockOptionLevel:
  0: 0
  1: 1
  2: 2
  3: 3

WorkLifeBalance:
  1: 

In [11]:
all_matters = compensation_matters + company_matters + carrer_growth_matters
column_name = ['Const'] + all_matters

X_with_constant = sm.add_constant(df[all_matters].values)
y = df['Attrition'].values
model = sm.OLS(y, X_with_constant).fit()
df_temp = pd.DataFrame({'Column Name': column_name, 'P Values': model.pvalues.round(5)})\
    .sort_values(by='P Values', ascending=True)\
    .reset_index(drop=True)\
    .drop(0)

df_temp[df_temp['P Values']<0.05]

,Column Name,P Values
1,StockOptionLevel,0.00000
2,EnvironmentSatisfaction,0.00000
3,OverTime,0.00000
4,YearsSinceLastPromotion,0.00104
5,JobSatisfaction,0.00113
6,TotalWorkingYears,0.00371
7,NumCompaniesWorked,0.00389
8,RelationshipSatisfaction,0.00488
9,YearsWithCurrManager,0.01276
10,YearsInCurrentRole,0.02390


In [12]:
initial_dataset = df.copy()
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])


In [13]:
X = df.drop('Attrition', axis=1)
y = df['Attrition']

In [14]:
sampler = SMOTE(random_state=42)
X_resampled, y_resampled = sampler.fit_resample(X, y)
X_train_resampled, X_test, y_train_resampled, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

## Modeling + Evaluation

In [15]:
list_precision = []
list_recall = []
list_f1 = []
list_accuracy = []
list_idx = []
model_saved = []

for idx, mat in enumerate(df_temp[df_temp['P Values']<0.05]['Column Name'].values):
    used_matters = df_temp[df_temp['P Values']<0.05]['Column Name'].values[:idx+1]
    algo = XGBClassifier(random_state=42, objective='binary:logistic',
                        # scale_pos_weight=y.value_counts()[0]/y.value_counts()[1],
                        )
    # algo = RandomForestClassifier(random_state=42)

    algo.fit(X_train_resampled[used_matters], y_train_resampled)
    y_pred = algo.predict(X_test[used_matters])
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    list_precision.append(precision)
    list_recall.append(recall)
    list_f1.append(f1)
    list_accuracy.append(accuracy)
    list_idx.append(idx+1)
    model_saved.append(algo)
    
df_result = pd.DataFrame({
    'column used': list_idx,
    'precision': list_precision,
    'recall': list_recall,
    'f1': list_f1,
    'accuracy': list_accuracy,
})


df_result.head(15)

,column used,precision,recall,f1,accuracy
0,1,0.650943,0.784091,0.711340,0.681818
1,2,0.674419,0.659091,0.666667,0.670455
2,3,0.693878,0.772727,0.731183,0.715909
3,4,0.790419,0.750000,0.769679,0.775568
4,5,0.845238,0.806818,0.825581,0.829545
5,6,0.862069,0.852273,0.857143,0.857955
6,7,0.858757,0.863636,0.861190,0.860795
7,8,0.877095,0.892045,0.884507,0.883523
8,9,0.889503,0.914773,0.901961,0.900568
9,10,0.879781,0.914773,0.896936,0.894886


In [16]:
best_idx = df_result['f1'].idxmax()
best_idx

8

In [17]:
used_features = df_temp[df_temp['P Values']<0.05]['Column Name'].values[:best_idx+1]
used_features

array(['StockOptionLevel', 'EnvironmentSatisfaction', 'OverTime',
       'YearsSinceLastPromotion', 'JobSatisfaction', 'TotalWorkingYears',
       'NumCompaniesWorked', 'RelationshipSatisfaction',
       'YearsWithCurrManager'], dtype=object)

In [18]:
used_numeric_features = [i for i in used_features if i in numerical_features]
used_numeric_features

['YearsSinceLastPromotion',
 'TotalWorkingYears',
 'NumCompaniesWorked',
 'YearsWithCurrManager']

In [19]:
fixed_scaler = MinMaxScaler()
fixed_scaler.fit(initial_dataset[used_numeric_features])

with open('./model/scaler.pkl', 'wb') as e:
    pickle.dump(fixed_scaler, e)

best_model = model_saved[best_idx]

with open('./model/model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [21]:
scaler = pickle.load(open('./model/scaler.pkl', 'rb'))
model = pickle.load(open('./model/model.pkl', 'rb'))

initial_dataset[used_numeric_features] = scaler.transform(initial_dataset[used_numeric_features])
X = initial_dataset.drop('Attrition', axis=1)
y = initial_dataset['Attrition']

y_pred = model.predict(X[used_features])
y_pred_proba = model.predict_proba(X[used_features])[:, 1]
f1 = f1_score(y, y_pred)
accuracy = accuracy_score(y, y_pred)

f1, accuracy

(0.9035812672176309, 0.9669187145557656)

In [34]:
y_pred[2], model.predict_proba(X[used_features])[2, y_pred[2]]

(np.int64(0), np.float32(0.9988574))